# DataDetective v.09

### Objective
Your goal is to guess a secret object chosen randomly from a list. You have a limited number of guesses to figure out what the object is.

### Gameplay
- You'll input your guesses through an interactive interface.
- After each guess, you'll receive feedback on whether you were right or wrong.
- If wrong, you're encouraged to try again until you either guess correctly or exhaust your attempts.

In [ ]:
## Student Todos
# 1. File-based values. This version has a static list of game values. Replace with a list stored in a file in local directory. Objects/Probabilities/Payouts
# 2. What happens if the file values are not present? Create a default set.
# 3. Create a process to storing the users results.
# 4. List the top ten results in a leaderboard.



In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from random import sample, choices
import numpy as np

class InteractiveGuessingGame:
    def __init__(self):
        self.objects = [
            "chalkboard or whiteboard", "desks and chairs", "textbooks",
            "notebooks or writing pads", "pens and pencils", "projector",
            "computer", "clock", "bulletin board", "maps or globes"
        ]
        # List of probabilities for each item being in a classroom
        self.probabilities = [
            0.9, 0.8, 0.5, 0.1, 0.2,
            0.9, 0.8, 0.5, 0.05, 0.02
        ]
        self.payout_multipliers = [
            .25, .5, 1, 3, 2,
            .25, .5, 1, 5, 20
        ]


        self.max_attempts = 5
        self.max_wager = 5.00
        self.max_payout = 300
        self.discount = 0.92

        self.losing_messages = [
            "Better luck next time, {name}! Keep trying!", "Not quite right, {name}. Think harder!",
            "{name}, are you even trying? Guess again!", "Oh no, {name}! That's not it. Try again!",
            "Keep going, {name}! You can do it!", "{name}, you're losing steam! Time to turn it around!"
        ]
        self.previous_message_index = -1

        self.reset_game()

    def reset_game(self):
        self.secret_objects = sample(self.objects, self.max_attempts)
        self.remaining_objects = self.objects.copy()  # Refresh the list of available objects
        self.guess_history = []
        self.correct_guesses = []
        self.incorrect_guesses = []
        self.other_guesses = []
        self.attempts = 0

    def check_guess(self, guess):
        normalized_guess = guess.lower()
        self.attempts += 1
        self.guess_history.append(normalized_guess)
        if normalized_guess in [obj.lower() for obj in self.remaining_objects]:
            self.remaining_objects.remove(guess)  # Remove guessed item from available options
        if normalized_guess in [obj.lower() for obj in self.secret_objects]:
            self.correct_guesses.append(normalized_guess)
            return True
        self.incorrect_guesses.append(normalized_guess)
        return False

    def get_new_losing_message(self, name):
        available_indices = [i for i in range(len(self.losing_messages)) if i != self.previous_message_index]
        new_message_index = np.random.choice(available_indices)
        self.previous_message_index = new_message_index
        return self.losing_messages[new_message_index].format(name=name)

    def get_results(self):
        correct = set(self.correct_guesses)
        incorrect = set(self.incorrect_guesses) - correct
        other = set(self.other_guesses) - correct - incorrect
        return correct, incorrect, other

    def calculate_payoff(self, wager):
        payoff = 0
        details = []
        for guess in self.correct_guesses:
            index = self.objects.index(guess)
            prob = self.probabilities[index]
            multiplier = self.payout_multipliers[index]
            amount = round(min(multiplier * wager * self.discount, self.max_payout),1)
            payoff += amount
            details.append((guess, prob, multiplier, amount))
        return payoff, details

def game_widget():
    game = InteractiveGuessingGame()
    name_input = widgets.Text(placeholder='Enter your name here...', description='Player Name:')
    guess_input = widgets.Dropdown(options=game.remaining_objects, description='Guess:')
    wager_input = widgets.Dropdown(options=[("$5.00 (max)", 5.00), ("$1.00", 1.00), ("$0.25", 0.25), ("$0.10", 0.10)], description='Wager:')
    button = widgets.Button(description="Check Guess")
    retry_button = widgets.Button(description="Retry", disabled=True)
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output()
            if not name_input.value:
                print("Please enter your name first!")
                return
            if wager_input.value > game.max_wager:
                print(f"Wager exceeds the maximum limit of ${game.max_wager}")
                return
            guess = guess_input.value
            game.check_guess(guess)
            guess_input.options = game.remaining_objects  # Update dropdown options
            if len(game.remaining_objects) > 0:
                guess_input.value = game.remaining_objects[0]  # Set the next item as default
            else:
                guess_input.value = None

            if game.attempts < game.max_attempts:
                print(f"Correct, {name_input.value}! The object {guess} is one of the secret objects." if guess.lower() in game.correct_guesses else game.get_new_losing_message(name_input.value))
                print(f"You have {game.max_attempts - game.attempts} more tries left.")
            else:
                correct, incorrect, other = game.get_results()
                payoff, details = game.calculate_payoff(wager_input.value)
                if not correct:
                    print("No matches made. Better luck next time")
                else:
                    print(f"Correct guesses: {correct}")
                print(f"No more attempts left, {name_input.value}. The objects were {', '.join(game.secret_objects)}.")
                print(f"Incorrect but on list: {incorrect}")
                print(f"Congratulations! You've won ${payoff:.2f}!")
                print("Payoff details:")
                for detail in details:
                    print(f"Guess: {detail[0]}, Probability: {detail[1]:.2f}, Multiplier: {detail[2]}, Payout: ${detail[3]:.2f}")
                guess_input.disabled = True
                button.disabled = True
                retry_button.disabled = False

    def on_retry_button_clicked(b):
        game.reset_game()
        guess_input.options = game.remaining_objects  # Reset dropdown options
        if len(game.remaining_objects) > 0:
            guess_input.value = game.remaining_objects[0]  # Set the next item as default
        else:
            guess_input.value = None
        guess_input.disabled = False
        button.disabled = False
        retry_button.disabled = True
        output.clear_output()
        print("New game started. Make your guesses!")

    button.on_click(on_button_clicked)
    retry_button.on_click(on_retry_button_clicked)

    display(name_input, guess_input, wager_input, button, output, retry_button)

game_widget()


Text(value='', description='Player Name:', placeholder='Enter your name here...')

Dropdown(description='Guess:', options=('chalkboard or whiteboard', 'desks and chairs', 'textbooks', 'notebook…

Dropdown(description='Wager:', options=(('$5.00 (max)', 5.0), ('$1.00', 1.0), ('$0.25', 0.25), ('$0.10', 0.1))…

Button(description='Check Guess', style=ButtonStyle())

Output()

Button(description='Retry', disabled=True, style=ButtonStyle())